In [1]:
## This notebook selects the wav samples based on the sound classes of interest

In [2]:
## Import packages
import os
import csv
import random
import math
import ntpath
import numpy as np
import pandas as pd
import time
from pathlib import Path
from scipy.io import wavfile
import sys

In [3]:
# Project directory
project_name = 'amsterdam_custom_samples'

In [4]:
# Set your serval data folder (should be correctly set already)
serval_data_folder = "../data"
project_data_folder = serval_data_folder + '/' + project_name

In [5]:
# location of wav samples to be used
# Youtube samples from enumerated csv files
input_balanced_wav_samples_enumerated_filepath   = serval_data_folder + "/csv_files/wav_samples_youtube_balanced_all_enumerated_and_labeled.csv"
input_unbalanced_wav_samples_enumerated_filepath = serval_data_folder + "/csv_files/wav_samples_youtube_unbalanced_all_enumerated_and_labeled.csv"
input_eval_wav_samples_enumerated_filepath       = serval_data_folder + "/csv_files/wav_samples_youtube_eval_all_enumerated_and_labeled.csv"

# Custom samples
input_custom_wav_samples_enumerated_filepath      = serval_data_folder + "/wav_samples_custom/wav_samples_custom_all_enumerated_and_labeled.csv"

In [6]:
# Select classes of interest
input_selected_classes_filepath = project_data_folder + '/csv_files/01_input_selected_classes.csv'
target_selected_classes_filepath = project_data_folder + '/csv_files/01_output_selected_wav_samples.csv'

In [7]:
# Load selected classes from csv
df_selected_classes = pd.read_csv(input_selected_classes_filepath, sep=",")

# Print
df_selected_classes

,label,mid,display_name
0,1000,/c/a_1000,Stilte
1,1001,/c/a_1001,556 gunshots
2,1002,/c/a_1002,Brommer Alarm
3,1003,/c/a_1003,Brommer
4,1004,/c/a_1004,Claxons
5,1005,/c/a_1005,Dichtklappende autodeuren
6,1006,/c/a_1006,"Luide mensen (schreeuwen, enz)"
7,1007,/c/a_1007,Motoren
8,1008,/c/a_1008,Terrasgeluide (stille mensen)
9,1009,/c/a_1009,Versterkte muziek


In [8]:
# Load youtube wav samples and classes
df_balanced_wav_samples_enumerated   = pd.read_csv(input_balanced_wav_samples_enumerated_filepath, sep=";")
df_unbalanced_wav_samples_enumerated = pd.read_csv(input_unbalanced_wav_samples_enumerated_filepath, sep=";")
df_eval_wav_samples_enumerated       = pd.read_csv(input_eval_wav_samples_enumerated_filepath, sep=";")

# Remove unnamed index column if it exists
df_balanced_wav_samples_enumerated   = df_balanced_wav_samples_enumerated.loc  [:, ~df_balanced_wav_samples_enumerated.columns.str.match('Unnamed')]
df_unbalanced_wav_samples_enumerated = df_unbalanced_wav_samples_enumerated.loc[:, ~df_unbalanced_wav_samples_enumerated.columns.str.match('Unnamed')]
df_eval_wav_samples_enumerated       = df_eval_wav_samples_enumerated.loc      [:, ~df_eval_wav_samples_enumerated.columns.str.match('Unnamed')]

# Combine
df_youtube_samples = df_balanced_wav_samples_enumerated.append(df_unbalanced_wav_samples_enumerated).append(df_eval_wav_samples_enumerated)

# Print
df_youtube_samples.head()

,label,mid,display_name,source,filename,filepath
0,500,/m/028v0c,Silence,wav_samples_youtube/bal,oh08bbDVFZM,../data/wav_samples_youtube/bal/vidoh08bbDVFZM...
1,435,/m/07pxg6y,Eruption,wav_samples_youtube/bal,oh08bbDVFZM,../data/wav_samples_youtube/bal/vidoh08bbDVFZM...
2,509,/t/dd00128,"Outside, urban or manmade",wav_samples_youtube/bal,oh08bbDVFZM,../data/wav_samples_youtube/bal/vidoh08bbDVFZM...
3,347,/m/01j4z9,Chainsaw,wav_samples_youtube/bal,xn_7_qB3L9M,../data/wav_samples_youtube/bal/vidxn_7_qB3L9M...
4,300,/m/07yv9,Vehicle,wav_samples_youtube/bal,xn_7_qB3L9M,../data/wav_samples_youtube/bal/vidxn_7_qB3L9M...


In [9]:
# Load custom wav samples and classes
df_custom_wav_samples_enumerated = pd.read_csv(input_custom_wav_samples_enumerated_filepath, sep=";")

# Remove unnamed index column if it exists
df_custom_wav_samples_enumerated = df_custom_wav_samples_enumerated.loc  [:, ~df_custom_wav_samples_enumerated.columns.str.match('Unnamed')]

# Print
df_custom_wav_samples_enumerated.head()

,label,mid,display_name,source,filename,filepath
0,1000,/c/a_1000,Silence,custom_amsterdam_sample,silence_02 (copy 3).wav,../data/wav_samples_custom/0_silence/silence_0...
1,1000,/c/a_1000,Silence,custom_amsterdam_sample,silence_05 (copy 4).wav,../data/wav_samples_custom/0_silence/silence_0...
2,1000,/c/a_1000,Silence,custom_amsterdam_sample,silence_03 (copy 4).wav,../data/wav_samples_custom/0_silence/silence_0...
3,1000,/c/a_1000,Silence,custom_amsterdam_sample,silence_05 (copy 5).wav,../data/wav_samples_custom/0_silence/silence_0...
4,1000,/c/a_1000,Silence,custom_amsterdam_sample,silence_04 (copy 5).wav,../data/wav_samples_custom/0_silence/silence_0...


In [10]:
# Combine
df_wav_samples_enumerated = df_youtube_samples.append(df_custom_wav_samples_enumerated)

In [11]:
# Filter all csv with interesting classes
df_selected_samples = df_wav_samples_enumerated.loc[df_wav_samples_enumerated.mid.isin(df_selected_classes.mid)]

In [12]:
# remove specific class 318
df_selected_samples = df_selected_samples[df_selected_samples.label != 318]

In [13]:
df_selected_samples

,label,mid,display_name,source,filename,filepath
0,1000,/c/a_1000,Silence,custom_amsterdam_sample,silence_02 (copy 3).wav,../data/wav_samples_custom/0_silence/silence_0...
1,1000,/c/a_1000,Silence,custom_amsterdam_sample,silence_05 (copy 4).wav,../data/wav_samples_custom/0_silence/silence_0...
2,1000,/c/a_1000,Silence,custom_amsterdam_sample,silence_03 (copy 4).wav,../data/wav_samples_custom/0_silence/silence_0...
3,1000,/c/a_1000,Silence,custom_amsterdam_sample,silence_05 (copy 5).wav,../data/wav_samples_custom/0_silence/silence_0...
4,1000,/c/a_1000,Silence,custom_amsterdam_sample,silence_04 (copy 5).wav,../data/wav_samples_custom/0_silence/silence_0...
...,...,...,...,...,...,...
3287,1009,/c/a_1009,Versterkte muziek,custom_amsterdam_sample,V.10.200102.125607.92.wav,../data/wav_samples_custom/v_versterkte_muziek...
3288,1009,/c/a_1009,Versterkte muziek,custom_amsterdam_sample,V.RGracht-0009.161956.wav,../data/wav_samples_custom/v_versterkte_muziek...
3289,1009,/c/a_1009,Versterkte muziek,custom_amsterdam_sample,V.RGracht-0004.171302.wav,../data/wav_samples_custom/v_versterkte_muziek...
3290,1009,/c/a_1009,Versterkte muziek,custom_amsterdam_sample,V-Roos07.160334.58.wav,../data/wav_samples_custom/v_versterkte_muziek...


In [14]:
display(df_selected_samples.groupby(['label', 'mid', 'display_name']).agg(['nunique']))

,,,source,filename,filepath
,,,nunique,nunique,nunique
label,mid,display_name,,,
1000,/c/a_1000,Silence,1,30,30
1001,/c/a_1001,556 gunshots,1,876,876
1002,/c/a_1002,Brommer Alarm,1,23,23
1003,/c/a_1003,Brommer,1,812,812
1004,/c/a_1004,Claxons,1,132,132
1005,/c/a_1005,Dichtklappende autodeuren,1,164,164
1006,/c/a_1006,"Luide mensen (schreeuwen, enz)",1,634,634
1007,/c/a_1007,Motoren,1,51,51


In [15]:
# sample down classes over 1000 samples
df = df_selected_samples
# modified : sample size based on proportions of group size
n = df.shape[0]
sample_df = df.groupby('display_name').apply(lambda x: x.sample(n=len(x) if len(x)<1000 else 1000))
# hk moet blijkbaar
sample_df.index.display_name = None

In [16]:
display(sample_df.groupby(['label', 'mid']).agg(['nunique']))

,,display_name,source,filename,filepath
,,nunique,nunique,nunique,nunique
label,mid,,,,
1000,/c/a_1000,1,1,30,30
1001,/c/a_1001,1,1,876,876
1002,/c/a_1002,1,1,23,23
1003,/c/a_1003,1,1,812,812
1004,/c/a_1004,1,1,132,132
1005,/c/a_1005,1,1,164,164
1006,/c/a_1006,1,1,634,634
1007,/c/a_1007,1,1,51,51


In [17]:
# Write csv out again to 
#df_selected_samples.to_csv(target_selected_classes_filepath, sep=';')
sample_df.to_csv(target_selected_classes_filepath, sep=';')